# Convert Sigmet files into Zarr

## Imports

In [1]:
import xradar as xd
import fsspec
import datatree
import pyproj
import holoviews as hv
import hvplot
import hvplot.xarray
from datetime import datetime
from wradlib.georef import epsg_to_osr, georeference
from cartopy import crs as ccrs
from sigmet2zarr.utils import create_query, check_if_exist, make_dir
from sigmet2zarr.task2zarr import raw2zarr

Now let's convert some sigmet files hosted in [IDEAM](https://registry.opendata.aws/ideam-radares/) AWS bucket.

In [2]:
# radar
radar_name = "Carimagua"
zarr_version = 2

# Store path to save radar datatree in Zarr format
zarr_store = f'../zarr/{radar_name}.zarr'
make_dir(zarr_store)

# Lest define some dates to convert
year, months, days = 2022, range(8, 9), range(9, 10)

In [ ]:
# data needs to be converted sequentially since zarr is using consolidated metadata
for month in months:                                                                                                                                                                                                                                                                                                              
    for day in days:
        # datetime object
        date_query = datetime(year=year, month=month, day=day)
        
        # Create AWS bucket string query
        query = create_query(date=date_query, radar_site=radar_name)
        str_bucket = 's3://s3-radaresideam/'

        # Using Fsspec to create a virtual connection with the S3 Bucket
        fs = fsspec.filesystem("s3", anon=True)

        # Glob the files within the bucket
        radar_files = sorted(fs.glob(f"{str_bucket}{query}*"))

        # loop through each file that will be converted and saved in zarr format. 
        # This needs to be done sequentially since data will be consolidated in the zarr store
        if radar_files:
            for i in radar_files[:100]: # looping over the first 100 files. You can just replace by radar_files.
                raw2zarr(i, store=zarr_store, mode='a', consolidated=True, 
                         append_dim='vcp_time', zarr_version=zarr_version,)

/data/keeling/a/alfonso8/miniconda3/envs/raw2zarr/lib/python3.12/site-packages/fsspec/registry.py:273: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/data/keeling/a/alfonso8/miniconda3/envs/raw2zarr/lib/python3.12/site-packages/xradar/io/backends/iris.py:234: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(decode_array(data, **kwargs))
/data/keeling/a/alfonso8/miniconda3/envs/raw2zarr/lib/python3.12/site-packages/xradar/io/backends/iris.py:234: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(decode_array(data, **kwargs))
/data/keeling/a/alfonso8/miniconda3/envs/raw2zarr/lib/python3.12/site-packages/xradar/io/backends/iris.py:234: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(decode_array(data, **kwargs))


## Read the radar datatree store in Zarr format

In [ ]:
dt_radar = datatree.open_datatree(zarr_store, engine='zarr')

In [ ]:
list(dt_radar.children)

We can now access each sweep by using a key-value method. Let's check the lowest elevation angle

In [ ]:
ds_05 = dt_radar['sweep_0'].ds
display(ds_05)

Xarray `Dataset` has `azimuth`, `range` and **`vcp_time`** cordiates. Which means that now we have multiple sweeps concatenated along this dimention allowing us to have a radar `timeseries`. We can create look at the horizontal radar reflectivity and create a plot using xarray plot built-in function

In [ ]:
ds_05.isel(vcp_time=0).DBZH.plot(cmap='ChaseSpectral', vmin=-10, vmax=50)

Before creating a radar plot we need to georeference the dataset. This can be done using `xradar.georeference` module

In [ ]:
ds_05 = xd.georeference.get_x_y_z(ds_05)
display(ds_05)

Now we can create a radial plot

In [ ]:
ds_05.isel(vcp_time=0).DBZH.plot(x="x", y="y", cmap="ChaseSpectral", vmin=-10, vmax=50)

In [ ]:
ref = ds_05.DBZH.hvplot.quadmesh(
    groupby="vcp_time",
    x="x",
    y="y",
    cmap="ChaseSpectral",
    clabel="Horizontal Reflectivity (dBZ)",
    title=f'Carimagua Radar',
    clim=(-20, 60),
    height=550,
    rasterize=True,
    width=700,widget_type="scrubber",
    widget_location="bottom",
)
ref